# Installing Libraries

In [2]:
!pip install  Upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from os.path import exists
import pandas as pd

In [6]:
df_path = "/content/train.csv.zip"

In [7]:
df = pd.read_csv(df_path)

In [9]:
df = df.sample(n= 19_000, random_state= 0)
df ["store"]= df["store"].astype(str)
df ["item"]= df["item"].astype(str)

In [12]:
df["date"]=pd.to_datetime(df["date"])
df.sort_values("date", inplace = True)
df.reset_index(inplace=True,drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [13]:
train = df[df["date"]< "2017-01-01"]
test = df[df["date"]> "2017-01-01"]

In [14]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

# Enrich Features


In [26]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv_type = CVType.time_series
)
enricher.fit(
    train_features,
    train_target,
    eval_set = [(test_features,test_target)]
)

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=6121eb43-b7a7-4763-acf3-ba7583a4d232
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


52 relevant feature(s) found with the search keys: ['date']


f_autofe_mul_efd66bbe,0.0754,100.0000,"-0.7432, -0.8075, 0.732",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_09412fe1,0.0290,100.0000,"0.1582, -1.1664, 0.2897",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_89c56a5f,0.0163,100.0000,"0.0561, 0.4596, 1.7001",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_0cc09d2d,0.0140,100.0000,"28.5891, -95.2196, 34.4594",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_08e89348,0.0084,100.0000,"21.6421, 72.7394, 95.348",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_0a5adf97,0.0066,100.0000,"0.0047, -0.3791, 0.0012",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.0034,100.0000,"-9.2029, 9.2563, 16.0565",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_events_date_year_cos1_9014a856,0.0030,100.0000,"0.815, 0.9751, -0.06",Upgini,Calendar data,Daily
f_autofe_div_a66c2b64,0.0029,100.0000,"0.0004, 0.0002, 0.0002",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_cf5dd7be,0.0024,100.0000,"-0.8845, -0.9689, 0.5683",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_ba625fe9,0.0023,100.0000,"-1.4255, 0.6176, 0.7935",Upgini,AutoFE: features from Calendar data,Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1657,21
Upgini,AutoFE: features from Calendar data,0.0080,7
Upgini,Calendar data,0.0034,5
Upgini,AutoFE: features from Markets data,0.0021,3
Upgini,World economic indicators,0.0010,8
Upgini,Markets data,0.0008,8


"Calendar data,Markets data",f_autofe_mul_efd66bbe,f_events_date_year_cos1_9014a856,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_div_09412fe1,f_events_date_year_cos1_9014a856,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_div_0a5adf97,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_92dac942,/
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_a66c2b64,f_events_date_week_sin1_847b5db1,f_financial_date_snp500_fdb64447,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
Calendar data,f_autofe_div_ba625fe9,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,/
Calendar data,f_autofe_mul_0afc7563,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,*



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,274.3901,172.4318,101.9582
Eval 1,3775,59.2940,376.3184,225.5075,150.8109


In [30]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
    train_features,
    train_target,
    eval_set = [(test_features,test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.297609,0.166783,0.130825
1,Eval 1,3775,59.2940,0.248317,0.145849,0.102467


In [36]:
enriched_train_features = enricher.transform(train_features,keep_input=True)
print(enriched_train_features)  # Add this line to inspect the value
print(type(enriched_train_features))  # Add this line to check the data type
enriched_test_features = enricher.transform(test_features,keep_input=True)


[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 9272.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

None
<class 'NoneType'>


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 728 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=ac7cccac-9dd4-4b60-a357-c3c775d669fc
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


In [39]:
if enriched_train_features is None:
    print("Warning: enricher.transform returned None. Using original features instead.")
    enriched_train_features = train_features.copy()  # Use original features as fallback
    # Convert enriched_train_features to pandas DataFrame if it's not already
    if not isinstance(enriched_train_features, pd.DataFrame):
        enriched_train_features = pd.DataFrame(enriched_train_features)

elif isinstance(enriched_train_features, dict):
    print("Converting enriched_train_features dictionary to DataFrame.")
    enriched_train_features = pd.DataFrame(enriched_train_features)

print(enriched_train_features)
print(type(enriched_train_features))

            date store item
0     2013-01-01     7    5
1     2013-01-01     4    9
2     2013-01-01     1   33
3     2013-01-01     3   41
4     2013-01-01     5   24
...          ...   ...  ...
15208 2016-12-31     6   26
15209 2016-12-31     8   14
15210 2016-12-31     1   37
15211 2016-12-31     5   21
15212 2016-12-31     1   32

[15213 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


In [40]:
model.fit (train_features,train_target)
pred = model.predict(test_features)
eval_metric(test_target.values,pred,"SMAPE")

[37.69703825674735]

In [42]:
model.fit (enriched_train_features,train_target)
enriched_pred = model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_pred,"SMAPE")

[37.69703825674735]